# Opdracht Regressie


De bedoeling van regressie is om op basis van een getraind model voorspellingen te doen van continue waarden.
Concreet komt het er op neer dat er een verband gezocht wordt tussen één of meerdere features en een zekere output variabele y. Dit verband is een (multidimensionale) curve van de vorm $y = \theta_n x_n + ... \theta_1 x_1 + \theta_0$. De vector X is de set van features en de set van $\theta$'s noemen we de parameters.

De doelstellingen van deze opdracht zijn:
- Data kunnen inlezen en verwerken met de pandas library.
- Eenvoudige statistische analyses uitvoeren op de data met als doel het detecteren van de consistenties van de data en het kunnen inschatten van de samenhang tussen de verschillende features en de output variabele.  
- Correct kunnen opsplitsen van de data in een training set en test set.
- Weten hoe je data normaliseert.
- Trainen van een regressiemodel.
- Kunnen evalueren van het getrainde regressiemodel via verschillende metrics zoals MAE en de $R^2$-score.
- Feature engineering toepassen: manueel of automatisch extra (hogere orde features) toevoegen.
- Kunnen detecteren van overfitting en underfitting.
- L1 en L2 regularisatie toepassen om te regelen tussen overfitting en underfitting.

In [23]:
%matplotlib inline
import datetime
import calendar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ML libraries inladen

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

# Voor statistische data visualisatie
import seaborn as sns; sns.set(color_codes=True)

## 1. Wijnkwaliteit

Van heel wat (rode en witte) portugese Vinho Verde wijnen(http://www.vinhoverde.pt/en/) werden een aantal chemische parameters opgemeten. Ook werd aan een panel van wijnexperten gevraagd om de wijnen een kwaliteitsscore op smaak te geven tussen 0 en 10.

Een overzicht van de features: 1- fixed acidity | 2 - volatile acidity | 3 - citric acid | 4 - residual sugar | 5 - chlorides | 6 - free sulfur dioxide | 7 - total sulfur dioxide | 8 - density | 9 - pH | 10 - sulphates |11 - alcohol

De output of target variabele is 'quality', de kwaliteitsscore (score tussen 0 en 10).

De bedoeling is om via meervoudige regressie zo betrouwbaar mogelijk de kwaliteitsscore van een wijn te kunnen voorspellen op basis van de chemische parameters ervan.

In [14]:
# Dataset vanuit een csv bestand inlezen in een pandas dataframe.
dataset = pd.read_csv('winequality-red.csv',';')
dataset.shape

(1599, 12)

In [15]:
# Eerste rijen van de dataset
dataset.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Statistische vooranalyse

- Geef een samenvattende tabel met beschrijvende centrum- en spreidingswaarden via de describe funtie van pandas.
- Ga na of de dataset ontbrekende waarden bevat.
- Is er op het eerste zicht sprake van uitschieters? Niettegenstaande dit voor multidimensionele gecorreleerde data niet volledig opgaat mag je bij deze opdracht uitschieters aanzien als featurewaarden die meer dan 5 standaardafwijkingen rond het gemiddelde gelegen zijn.
- Maak een correlatie heatmap. Welke features lijken het meest invloed te hebben op de kwaliteitsscore? Zijn er features die onderling sterk gecorreleerd zijn. Als je 5 features zou willen weglaten, welke zouden dit zijn? Verklaar waarom. Bij deze vraag is de verklaring belangrijk.
- Maak een pairplot, bekijk deze aandachtig en formuleer enkele conclusies. Zijn er verbanden te leggen tussen wat je ziet op de pairplot en op de correlatie heatmap?

In [1]:
# Weglaten van de uitschieters:
# Normaal gezien alleen maar doen als je weet dat de uitschieters het gevolg zijn van 
# inconsistenties of fouten.



In [2]:
# Lineaire correlatie tussen de features - heatmap


In [3]:
# Pairplot (=scatterplot). (Dit kan wat tijd vragen om te genereren)
# Formuleer conclusies


### Opsplitsen van de dataset

Splits de dataset op in features X en targets y. Vervolgens kan je de dataset opsplitsen in een training set en een test set. Gebruik hiervoor de functie train_test_split van Sklearn (http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Zorg dat er 30 procent van de dataset in de test set terecht komt.

Normaliseer de training set en de test set met een standardscaler. Probeer later ook met een robustscaler.
Verklaar wat het verschil is tussen de standardscaler en de robustscaler.
De labels hoeven niet genormaliseerd te worden.

In [5]:
# Dataset opsplitsen in features en target


In [6]:
# Opsplitsen in training set en test set


In [7]:
# Normaliseren van de training set en de test set


## Trainen van het regressiemodel

Train een regressiemodel via linear_model.LinearRegression(). Print vervolgens de coëfficiënten van het model. 

In [8]:
# Trainen van het regressiemodel


In [9]:
# print de coëfficiënten en de intercept van het model


### Testen en voorspellen
Voorspel de kwaliteitsscore van een wijn met chemische parameters:
									
fixed_acidity: 7.3 | 
volatile_acidity: 0.70 | citric_acid: 0.00 | residual_sugar: 1.9 | chlorides: 0.079 | free_sulfur_dioxide: 11.0 | total_sulfur_dioxide: 34.0 | density: 0.9978 | pH: 3.51 | sulphates: 0.58 | alcohol: 9.4

In [10]:
# Voorspelling van de kwaliteitsscore van de wijn.


Evalueer het regressiemodel op basis van de MAE, MSE en de $R^2$-score. Gebruik hiervoor de test set. Formuleer een aantal conclusies uit deze scores.

In [11]:
# Evaluatie van het model via de MAE, MSE en de R2-score


### Modeloptimalisatie
Probeer de performantie van het model te verbeteren. Zorg er met andere woorden voor dat de MAE en de MSE verlaagd worden en de $R^2$ score toeneemt.
Mogelijke opties zijn:
- Toevoegen van hogere orde features (manueel). Motiveer waarom je bepaalde hogere orde features zou gebruiken. Kijk hiervoor bijvoorbeeld naar de pairplot of de correlatie heatmap (ga op zoek naar niet-lineariteiten).
- Extra features afleiden uit de bestaande features.
- Toepassen van regularisatie via Ridge regression en Lasso regression. Ga telkens op zoek naar een goede waarde voor alpha. 
- Ga telkens na of er overfitting over underfitting optreedt. Doe dit door de performantie op de training set te vergelijken met de performantie op de test set. 
- Voeg automatisch hogere orde features toe via sklearn.preprocessing.PolynomialFeatures. Ga niet hoger dan 3de of 4de orde. Het aantal features neemt namelijk exponentiëel toe met de orde met als gevolg dat het model niet meer binnen een aanneembare tijd te trainen valt.

Motiveer telkens de gemaakte keuze en evalueer het model met de test set. Wat is de beste performantie die je haalt? **Bespreek het eindresultataat**.

In [14]:
# Modeloptimalisatie en Hyperparameter tuning


In [15]:
### Trainen en testen met gereduceerde features


## 2. Bierconsumptie

In het bestand *beerconsumption.csv* vind je een dataset met gegevens over de dagelijkse bierconsumptie in en een bepaalde stad (van het jaar 2015) in functie van de datum en de weersomstandigheden.
Bedoeling is om een regressiemodel te vinden dat een zo accuraat mogelijke voorspelling kan doen van de te verwachten bierconsumptie op een zekere dag met bepaalde te verwachten weersomstandigheden.

De verschillende features omvatten:
- date: datum
- average_temp: gemiddelde temperatuur van die dag in graden Celcius
- min_temp: minimum temperatuur van die dag in graden Celcius
- max_temp: maximum temperatuur van die dag in graden Celcius
- precipitation: de hoeveelheid neerslag in mm
- weekend: binaire variabele (0/1) die aanduidt of het al dan niet weekend is

De te voorspellen variabele (target) is 'consumption', de totale dagelijkse bierconsumptie in liter.


**Opdracht:**

- Controleer de dataset op inconsistenties: ontbrekende waarden, foutieve en extreme waarden
- Voeg twee kolommen toe aan de dataset. Noem deze kolommen 'month' en 'day'. In de ene kolom komt de naam van de maand (January, February, March, ...), in de andere de naam van de dag van de week (Monday, Tuesday, ...). Zoek online hoe je uit de datum de maandnaam en dagnaam kan halen. Verwijder vervolgens de 'date' kolom. Deze hebben we niet meer nodig.
- Vooranalyse: visualiseer de bierconsumptie per maand, bijvoorbeeld via boxplots (https://seaborn.pydata.org/generated/seaborn.boxplot.html). In welke maand wordt het meest bier geconsumeerd en in welke maand het minst? Visualiseer ook de bierconsumptie per dag. Op welke dagen van de week wordt het meest bier geconsumeerd en op welke dag het minst.

- Pas one-hot encoding toe op de features 'month' en 'day'. Zo worden categorische variabelen omgezet naar numerieke one-hot variabelen. Meer info over one-hot encoding is te vinden op https://towardsdatascience.com/choosing-the-right-encoding-method-label-vs-onehot-encoder-a4434493149b.

- Splits op in features en targets en maak vervolgens een training set en test set aan. Zorg ervoor dat er 100 samples in de test set zitten.

- Normaliseer de features. Categorische variabelen hoeven niet genormaliseerd te worden. Baseer je op de histogrammen om te beslissen welke normalisatie je zal toepassen (Standardscaler, MinMaxscaler of Robustscaler). De MinMaxscaler laat one-hot variabelen onveranderd.

- Train een regressiemodel

- Test het getraind model op de test set

- Doe optimalisaties en hyperparameter tuning om de performantie van het model (op de test set) te verhogen. Maak een grafiek van de r²-score in functie van de regularisatieparameter alpha.

- Bespreek het finaal bekomen model. Wat is de R²-score? Wat is de betekenis ervan? Wat zegt de bekomen MAE? Ga ook na welke twee features de grootste invloed hebben op de bierconsumptie.

- Zijn er features die je mag weglaten? Met andere woorden, zijn er overbodige features? Argumenteer.

- Voorspel de bierconsumptie op een zondag in juni bij een gemiddelde temperatuur van 28.64 graden, een min_temp van 25.22 en een max_temp van 31.08. Er is geen neerslag.



In [16]:
# Uitwerking opdracht bierconsumptie

